In [4]:
from pydma.databases import OneFLDb
import pandas as pd

In [9]:
pain = OneFLDb('rsr', database='DWS_PAIN_OMOP')

In [49]:
schema = 'preload'
table = 'condition_occurrence'
pre_tables = pain.list_tables(in_schema=[schema])
catalog_1 = 'dws_pain_omop'
catalog_2 = 'dws_omop'

for table in pre_tables.Table:
    with pain.engine.connect() as con:
        pain_t = pd.read_sql(f'select top 0 * from dws_pain_omop.{schema}.{table}', con)
        pain_dt = pd.read_sql(f"SELECT COLUMN_NAME, DATA_TYPE FROM {catalog_1}.INFORMATION_SCHEMA.COLUMNS where TABLE_NAME = '{table}' and TABLE_SCHEMA = '{schema}'", con)
        omop_t = pd.read_sql(f'select top 0 * from dws_omop.{schema}.{table}', con)
        omop_dt = pd.read_sql(f"SELECT COLUMN_NAME, DATA_TYPE FROM {catalog_2}.INFORMATION_SCHEMA.COLUMNS where TABLE_NAME = '{table}' and TABLE_SCHEMA = '{schema}'", con)

    compare = pd.DataFrame({'pain':pain_t.columns, 
                            'pain_dt': pain_dt.DATA_TYPE, 
                            'omop': omop_t.columns, 
                            'omop_dt': omop_dt.DATA_TYPE})
    compare['q_name'] = compare.apply(lambda row: row.pain == row.omop, 1)
    compare['q_datatype'] = compare.apply(lambda row: row.pain_dt == row.omop_dt, 1)

    if all(compare.q_name) and all(compare.q_datatype): 
        print(f'table {table} OK')
        pass
    else: 
        print(table, compare)

CONDITION_OCCURRENCE                              pain   pain_dt                           omop  \
0                       person_id       int                      person_id   
1            condition_concept_id       int           condition_concept_id   
2            condition_start_date      date           condition_start_date   
3        condition_start_datetime  datetime       condition_start_datetime   
4              condition_end_date      date             condition_end_date   
5          condition_end_datetime  datetime         condition_end_datetime   
6       condition_type_concept_id       int      condition_type_concept_id   
7                     stop_reason   varchar                    stop_reason   
8                     provider_id       int                    provider_id   
9             visit_occurrence_id       int            visit_occurrence_id   
10                visit_detail_id       int                visit_detail_id   
11         condition_source_value   varchar